# Refatoração

**Prática! Aplicando, estudando e pesquisando as mudanças e novas alternativas sugeridas pelo Meigarom.**

## Agenda 
        1 - Breve Resumo Análise Exploratória
        2 - Breve Resumo Data Cleaning original
        3 - Meigarom Way of Cleaning
            3.1 PE.OLX
            3.2 REDEIMOVEISPE
                3.2.1 Preço
            3.3 VIVAREAL    
                3.3.1 Preço
                3.3.2 Locations
            3.4 ZAPIMOVEIS
            3.5 Unindo DataFrames

In [1]:
import pandas as pd
import re

## 1 - Breve Resumo Análise Exploratória 
A finalidade era entender a estrutura dos dados e avaliar quais mudanças deveriam ser feitas.

Com 3 arquivos para explorar e limpar, tinha decidido executar esses processos separadamente, imaginando que as diferenças entre os arquivos exijiriam abordagens diferentes (principalmente na limpeza).

**Identificando os desafios.**

As tarefas se resumem a: 
    
    1 Separar os dados por bairro. Assim teremos uma noção melhor sobre
      eles, principalmente por a localização ser um fator importante no 
      preço de um apartamento.
    2 Transformar e criar as colunas numéricas, para podermos fazer os 
      cálculos (tanto estatísticos quanto machine learnicos) com seus 
      valores.

Para isso devemos entender como os dados estão estruturados e pensar em como modificá-los. Como resultado, observei que:

### pe.olx

    "Todas as colunas numéricas estão no formato object (str, dict ou list). 
    Teremos que limpar e converter futuramente, sem contar com a separação
    dos dados da coluna ['other']."
    
    "E também temos alguns dados ausentes para lidar."

### redeimoveispe
    
    Problemas similares, mas sem dados ausentes
    
### vivareal
    
    A grande maioria dos dados são duplicatas

In [4]:
df_olx = pd.read_csv('0 - Apartamentos Dados (Bruto).csv', index_col=0)
df_olx.head()

,prices,locations,other
0,R$ 750.000,"Recife, Boa Viagem",3 quartos | 130m² | 1 vaga
1,R$ 200.000,"Jaboatão dos Guararapes, Candeias",2 quartos | 61m² | 1 vaga
2,R$ 300.000,"Recife, Cordeiro",2 quartos | 50m² | Condomínio: R$ 1 | 1 vaga
3,R$ 128.000,"Paulista, Nossa Senhora do Ó",2 quartos | 63m² | 1 vaga
4,R$ 125.000,"Recife, Areias",2 quartos | 62m² | Condomínio: R$ 140 | 1 vaga


In [20]:
df_redeimoveis = pd.read_csv('0.1 - Apartamentos Dados (Bruto).csv', index_col=0)
df_redeimoveis.head()

,prices,locations,other
0,R$ 80.000,Campo Grande,2quartos2banheiros1vaga67m²
1,R$ 130.000,Boa Vista,"1quarto1banheiro 58,40m²"
2,R$ 130.000,Linha do Tiro,2quartos1banheiro 1vaga65m²
3,R$ 132.000R$ 630/ano,Várzea,2quartos1banheiro 1vaga50m²
4,R$ 140.000,Tejipió,2quartos1banheiro 80vagas41m²


## 2 - Breve Resumo Data Cleaning Original

### pe.olx
    
        Removendo NaN. Quantia pequena, simplesmente eliminei.
    
    Preço:
        Removi os "R$", os pontos "." e converti o tipo object para numérico.
    
    Localização:
        Separei na vírgula para obter os bairros.
    
    other (Quarto, Tamanho, Garagem):
        Separei os elementos pelo "|". Porém essa divisão resultava vezes em 3 elementos,
        vezes em 4. O que me levou a "catar" os valores desejados quase que um por um 
        (testando, errando, encontrando mais problemas e testando novamente, 
        até conseguir, na próxima seção "other*" há um exemplo em código).
        Em outras palavras o Meigarom me mostrou uma maneira mais simples e rápida de 
        obter os dados necessários.
        
### redeimoveispe
    
    Preço:
        Também removi os "R$" e os pontos ".". Mas por haver valores númericos (que não 
        representavam os preços) junto a texto, criei um algoritmo muito louco kkkkkk
        para eliminar as anomalias. O pensamento inicial dele segue abaixo, o próximo
        passo foi vasculhar pelos índices e eliminá-los.
        
```python
for item in data.price:
    try:
        int(item)
    except:
        delete_data.append(item) # type = list
```
<br>
        
        other* (Quarto, Banheiro, Tamanho, Garagem):
            Deu menos trabalho, mas ainda assim ocupou bastante tempo com testes e erros.
            Basicamente, utilizei variações deste formato para alcançar os resultados desejados:
            
_Inputs:_ **"2quartos2banheiros1vaga67m²"** -----> **"2quartos1banheiro 80vagas41m²"** ----->  **...**

```python            
data.other.apply(lambda x: int(x.split('q')[0]))
```
_Outputs:_ **"2"** -------> **"2"** ------> **...**

```python
data.other.apply(lambda x: int(float(x.split('vaga')[1].replace('m²','').replace('s','').replace(',','.'))))
```
_Outputs:_ **"67"** -----> **"41"** ----->  **...**
     

### vivareal
_"A grande esmagadora e imensa maioria dos dados são duplicatas ('-' )"_

_"As localizações são bem específicas então não poderia haver 299 apartamentos em "Avenida Boa Viagem, 1616 - Boa Viagem, Recife - PE"_

Por haver apenas 36 dados que não eram duplicatas, abandonei esse dataset. Mas aqui vou limpar mesmo assim, apenas pela diversão kkkk

## 3 - Meigarom Way of Cleaning 

ps: acabei e percebi que usei sqft(square foot) ao invés de square meter (-.-' ) kkkk. Bom, não sendo o foco do exercício, abstrai e finge demência kkkk.

### 3.1 PE.OLX
    
        Eliminar NaN
        
    other:
        Dividir no "|" e utilizar Regex para obter os dados.
        
    Preço:
        Remover os "R$", os pontos "." e converter o tipo object para numérico.
    
    Localização:
        Separar na vírgula para obter os bairros.   

In [6]:
print(df_olx.isna().sum())
df_olx.dropna(inplace=True) # Get rid of NaN
df_olx.head()

prices       82
locations     2
other         2
dtype: int64


,prices,locations,other
0,R$ 750.000,"Recife, Boa Viagem",3 quartos | 130m² | 1 vaga
1,R$ 200.000,"Jaboatão dos Guararapes, Candeias",2 quartos | 61m² | 1 vaga
2,R$ 300.000,"Recife, Cordeiro",2 quartos | 50m² | Condomínio: R$ 1 | 1 vaga
3,R$ 128.000,"Paulista, Nossa Senhora do Ó",2 quartos | 63m² | 1 vaga
4,R$ 125.000,"Recife, Areias",2 quartos | 62m² | Condomínio: R$ 140 | 1 vaga


Uau! As 3 células abaixo executam o trabalho de toda a seção [1.3 - ['Quarto'] ['Tamanho'] ['Garagem']](https://nbviewer.jupyter.org/github/pauloreis-ds/Projetos/blob/master/Previs%C3%A3o%20-%20Pre%C3%A7o%20Apartamentos%20Recife/notebooks/2%20Data%20Cleaning%20Im%C3%B3veis.ipynb) do notebook principal. Na época, separei a captura dos valores por coluna, testando operações em cada uma delas individualmente até conseguir. Já este formato diminui bastante o tempo de execução:

    A grande diferença está na flexibilidade oferecida pela Regex (e pelo conhecimento de funções como .where())
    por facilitar e abrangir a área de busca, o que transporta determinados problemas para pontos em que conseguimos
    tratá-los com mais facilidade, como por exemplo: aqui, os valores "0" em ['bedrooms'] são derivados de antigos
    "0m²", os quais antes tive que enfrentar inicialmente como um problema à parte, mas agora é possível removê-los
    durante a análise estatística (outliers) ou até mesmo substituí-los utilizando conhecimento da área...
    
    Minha solução antiga me levou a realizar vários procedimentos para identificar qual funcionava em cada coluna.
    O que, consequentemente, elevou o tempo do processo.

In [7]:
other = df_olx['other'].str.split('|', expand=True)
other.columns = ['bedrooms','sqft','condominiums','garages']
other.head()

,bedrooms,sqft,condominiums,garages
0,3 quartos,130m²,1 vaga,None
1,2 quartos,61m²,1 vaga,None
2,2 quartos,50m²,Condomínio: R$ 1,1 vaga
3,2 quartos,63m²,1 vaga,None
4,2 quartos,62m²,Condomínio: R$ 140,1 vaga


In [8]:
# .where -> Replace values where the condition is False.
other[['condominiums','garages']] = other[['condominiums','garages']].where(other['garages'].notnull(), 
                                                                            other[['garages','condominiums']].values)
other.head()

,bedrooms,sqft,condominiums,garages
0,3 quartos,130m²,None,1 vaga
1,2 quartos,61m²,None,1 vaga
2,2 quartos,50m²,Condomínio: R$ 1,1 vaga
3,2 quartos,63m²,None,1 vaga
4,2 quartos,62m²,Condomínio: R$ 140,1 vaga


In [9]:
# Regex to only get numbers
other['bedrooms'] = other['bedrooms'].str.extract("(\d+)", expand=False)
other['sqft'] = other['sqft'].str.extract("(\d+)", expand=False)
other['garages'] = other['garages'].str.extract("(\d+)", expand=False)
other['condominiums'] = other['condominiums'].str.extract("(\d+)", expand=False)
other.head()

,bedrooms,sqft,condominiums,garages
0,3,130,NaN,1
1,2,61,NaN,1
2,2,50,1,1
3,2,63,NaN,1
4,2,62,140,1


**Agora que passamos pela parte mais complexa podemos concatenar as tabelas e finalizar essa limpeza do mesmo modo que fiz no notebook principal.**

In [10]:
df_olx = pd.concat([df_olx.drop(columns='other'), other], axis=1)
df_olx.head()

,prices,locations,bedrooms,sqft,condominiums,garages
0,R$ 750.000,"Recife, Boa Viagem",3,130,NaN,1
1,R$ 200.000,"Jaboatão dos Guararapes, Candeias",2,61,NaN,1
2,R$ 300.000,"Recife, Cordeiro",2,50,1,1
3,R$ 128.000,"Paulista, Nossa Senhora do Ó",2,63,NaN,1
4,R$ 125.000,"Recife, Areias",2,62,140,1


In [11]:
df_olx['prices'] = df_olx['prices'].apply(lambda x: int(x.split()[1].replace('.','').replace('R$','')))
df_olx['locations'] = df_olx['locations'].apply(lambda x: x.split(',')[-1])
df_olx.head()

,prices,locations,bedrooms,sqft,condominiums,garages
0,750000,Boa Viagem,3,130,NaN,1
1,200000,Candeias,2,61,NaN,1
2,300000,Cordeiro,2,50,1,1
3,128000,Nossa Senhora do Ó,2,63,NaN,1
4,125000,Areias,2,62,140,1


### 3.2 REDEIMOVEISPE
    other:
        Expressões regulares novamente facilitando o tratamento dos dados.
    
    Prices:
        Remover "R$" e utilizar re para caputar valores.
        

In [21]:
df_redeimoveis.head()

,prices,locations,other
0,R$ 80.000,Campo Grande,2quartos2banheiros1vaga67m²
1,R$ 130.000,Boa Vista,"1quarto1banheiro 58,40m²"
2,R$ 130.000,Linha do Tiro,2quartos1banheiro 1vaga65m²
3,R$ 132.000R$ 630/ano,Várzea,2quartos1banheiro 1vaga50m²
4,R$ 140.000,Tejipió,2quartos1banheiro 80vagas41m²


Expressões Regulares podem ser complicadas rsrsr... em [53:30](https://www.youtube.com/watch?v=cCpnbD7kpCQ&t=3210s) o Meigarom demonstra essa característica delas.

In [22]:
df_redeimoveis['bedrooms'] = df_redeimoveis['other'].apply(lambda x: re.findall('[0-9]+quartos?', x))
df_redeimoveis['bathrooms'] = df_redeimoveis['other'].apply(lambda x: re.findall('[0-9]+banheiros?', x))
df_redeimoveis['garages'] = df_redeimoveis['other'].apply(lambda x: re.findall('[0-9]+vagas?', x))
df_redeimoveis['sqft'] = df_redeimoveis['other'].apply(lambda x: re.findall('[0-9]+,?[0-9]+m²', x))
df_redeimoveis.head()

,prices,locations,other,bedrooms,bathrooms,garages,sqft
0,R$ 80.000,Campo Grande,2quartos2banheiros1vaga67m²,[2quartos],[2banheiros],[1vaga],[67m²]
1,R$ 130.000,Boa Vista,"1quarto1banheiro 58,40m²",[1quarto],[1banheiro],[],"[58,40m²]"
2,R$ 130.000,Linha do Tiro,2quartos1banheiro 1vaga65m²,[2quartos],[1banheiro],[1vaga],[65m²]
3,R$ 132.000R$ 630/ano,Várzea,2quartos1banheiro 1vaga50m²,[2quartos],[1banheiro],[1vaga],[50m²]
4,R$ 140.000,Tejipió,2quartos1banheiro 80vagas41m²,[2quartos],[1banheiro],[80vagas],[41m²]


Funcionou! agora só precisamos remover as string desnecessárias.

In [23]:
print(df_redeimoveis['bedrooms'].apply(lambda x: str(x)).unique(),'\n')
print(df_redeimoveis['bathrooms'].apply(lambda x: str(x)).unique(),'\n')
print(df_redeimoveis['garages'].apply(lambda x: str(x)).unique())

["['2quartos']" "['1quarto']" "['3quartos']" "['4quartos']" "['5quartos']"] 

["['2banheiros']" "['1banheiro']" "['3banheiros']" '[]' "['4banheiros']"
 "['5banheiros']" "['34banheiros']" "['6banheiros']"] 

["['1vaga']" '[]' "['80vagas']" "['2vagas']" "['3vagas']" "['4vagas']"]


In [24]:
df_redeimoveis['bedrooms'] = df_redeimoveis['bedrooms'].apply(lambda x: int(x[0].replace('quarto','').replace('s','')))

df_redeimoveis['bathrooms'] = df_redeimoveis['bathrooms'].apply(lambda x: int(x[0].replace('banheiro','').replace('s',''))
                                                                if len(x)>0 else 0)

df_redeimoveis['garages']=  df_redeimoveis['garages'].apply(lambda x: int(x[0].replace('vaga','').replace('s',''))
                                                            if len(x)>0 else 0)

df_redeimoveis['sqft'] = df_redeimoveis['sqft'].apply(lambda x: float(x[0].replace('m²','').replace(',','.')))

df_redeimoveis.drop(columns='other', inplace=True)
df_redeimoveis.head()

,prices,locations,bedrooms,bathrooms,garages,sqft
0,R$ 80.000,Campo Grande,2,2,1,67.0
1,R$ 130.000,Boa Vista,1,1,0,58.4
2,R$ 130.000,Linha do Tiro,2,1,1,65.0
3,R$ 132.000R$ 630/ano,Várzea,2,1,1,50.0
4,R$ 140.000,Tejipió,2,1,80,41.0


### 3.2.1 Preço 
Esta coluna requer uma atenção especial, utilizar apenas os métodos básicos de Regex não é o suficiente. Mas talvez com mais experiência e conhecimento sobre as expressões, elas possam ser um forte apoio aqui. 

Note que em uns elementos vemos " _R\$ 190.000Preço de locaçãoSob consulta_ " e em outros o " _Preço de vendaSob consultaR\$ 1.450/mês_ ".

In [25]:
df_redeimoveis['prices'].unique()[:20]

array(['R$ 80.000', 'R$ 130.000', 'R$ 132.000R$ 630/ano', 'R$ 140.000',
       'R$ 145.000R$ 720/ano', 'R$ 150.000', 'R$ 160.000', 'R$ 168.000',
       'R$ 170.000', 'Preço de vendaSob consultaR$ 1.450/mês',
       'R$ 174.000', 'R$ 180.000', 'R$ 182.000', 'R$ 186.000',
       'R$ 190.000', 'R$ 190.000Preço de locaçãoSob consulta',
       'R$ 190.000R$ 1.300/mês', 'R$ 195.000', 'R$ 198.990', 'R$ 199.000'],
      dtype=object)

In [26]:
df_redeimoveis['prices'] = df_redeimoveis['prices'].apply(lambda x: x.split("R$")[1].replace('.',''))

for item in df_redeimoveis['prices'].unique():
    try:
        int(item)
    except:
        print(item)

 1450/mês
 190000Preço de locaçãoSob consulta
 255000Preço de locaçãoSob consulta
 1750/mês
 270000Preço de locaçãoSob consulta
 290000Preço de locaçãoSob consulta
 1000/mês
 1250/mês
 350000Preço de locaçãoSob consulta
 2000/mês
 360000Preço de locaçãoSob consulta
 2200/mês
 500000Preço de locaçãoSob consulta
 1700/mês
 700000Preço de locaçãoSob consulta
 900000Preço de locaçãoSob consulta
 980000Preço de locaçãoSob consulta
 1350000Preço de locaçãoSob consulta


In [27]:
df_redeimoveis['prices'] = df_redeimoveis['prices'].apply(lambda x: int(re.findall("[\d]+",x)[0]))
df_redeimoveis.head()

,prices,locations,bedrooms,bathrooms,garages,sqft
0,80000,Campo Grande,2,2,1,67.0
1,130000,Boa Vista,1,1,0,58.4
2,130000,Linha do Tiro,2,1,1,65.0
3,132000,Várzea,2,1,1,50.0
4,140000,Tejipió,2,1,80,41.0


### 3.3 VIVAREAL
        Deletar duplicatas
        
    other:
        Regex
        
    Preço:
        Remover os "R$", os pontos "." e converter o tipo object para numérico.

    Localização:
        Wooow!!! \(*O*)/ Regexxxx!!

In [28]:
vivareal = pd.read_csv('0.2 - Apartamentos Dados (Bruto).csv', index_col=0)
vivareal.drop_duplicates(inplace=True)
vivareal.head()

,prices,locations,other
0,R$ 1.700.000,"Avenida Dezessete de Agosto, 2720 - Casa Fort...",207 Área m² 4 Quartos 4 Suíte...
1,Sob Consulta,"Rua Eurico de Souza Leão, 541 - Cordeiro, Rec...",55-73 Área m² 1-3 Quartos 1 B...
2,R$ 780.000,"Rua Wilfrid Rússel Shorto, 61 - Boa Viagem, R...",250 Área m² 4 Quartos 2 Suíte...
3,R$ 450.000,"Estrada do Encanamento, 1342 - Casa Forte, Re...",82 Área m² 3 Quartos 1 Suíte ...
4,R$ 350.000,"Rua Doutor Luiz Inácio Pessoa de Melo, 81 - B...",70 Área m² 2 Quartos 1 Suíte ...


In [29]:
vivareal.other.unique()[:5]

array(['   207   Área m²     4   Quartos     4   Suítes     5   Banheiros     3   Vagas   ',
       '   55-73   Área m²     1-3   Quartos     1   Banheiro     1-2   Vagas   ',
       '   250   Área m²     4   Quartos     2   Suítes     4   Banheiros     1   Vaga   ',
       '   82   Área m²     3   Quartos     1   Suíte     3   Banheiros     2   Vagas   ',
       '   70   Área m²     2   Quartos     1   Suíte     2   Banheiros     1   Vaga   '],
      dtype=object)

In [30]:
vivareal['sqft'] = vivareal['other'].apply(lambda x: re.findall('[(\d)]+-?[(\d)]+', x)[0])
vivareal['bedrooms'] = vivareal['other'].apply(lambda x: re.findall('[(\d)]+-?[0-9]?[0-9]?', x)[1])
vivareal['bathrooms'] = vivareal['other'].apply(lambda x: re.findall('[(\d)]+-?[0-9]?[0-9]?', x)[-2])
vivareal['garages'] = vivareal['other'].apply(lambda x: re.findall('[(\d)]+-?[0-9]?[0-9]?', x)[-1])
vivareal.head()

,prices,locations,other,sqft,bedrooms,bathrooms,garages
0,R$ 1.700.000,"Avenida Dezessete de Agosto, 2720 - Casa Fort...",207 Área m² 4 Quartos 4 Suíte...,207,4,5,3
1,Sob Consulta,"Rua Eurico de Souza Leão, 541 - Cordeiro, Rec...",55-73 Área m² 1-3 Quartos 1 B...,55-73,1-3,1,1-2
2,R$ 780.000,"Rua Wilfrid Rússel Shorto, 61 - Boa Viagem, R...",250 Área m² 4 Quartos 2 Suíte...,250,4,4,1
3,R$ 450.000,"Estrada do Encanamento, 1342 - Casa Forte, Re...",82 Área m² 3 Quartos 1 Suíte ...,82,3,3,2
4,R$ 350.000,"Rua Doutor Luiz Inácio Pessoa de Melo, 81 - B...",70 Área m² 2 Quartos 1 Suíte ...,70,2,2,1


### 3.3.1 Preço
Não tanto, mas um pouco complexo.

In [31]:
vivareal.prices.unique()[:5]

array(['      R$ 1.700.000    ', '    Sob Consulta   ',
       '      R$ 780.000    ', '      R$ 450.000    ',
       '      R$ 350.000    '], dtype=object)

In [32]:
vivareal['prices'] = vivareal['prices'].apply(lambda x: int(x.replace("R$", "").replace(".", ""))
                                              if x.replace("R$", "").replace(".", "").strip().isnumeric() else None )

### 3.3.2 Locations
**Wooow!!! \\(\*-\*)/ Regexxxx!!**

In [33]:
vivareal.drop(columns='other', inplace=True)
vivareal.locations.unique()[:5]

array([' Avenida Dezessete de Agosto, 2720 - Casa Forte, Recife - PE ',
       ' Rua Eurico de Souza Leão, 541 - Cordeiro, Recife - PE ',
       ' Rua Wilfrid Rússel Shorto, 61 - Boa Viagem, Recife - PE ',
       ' Estrada do Encanamento, 1342 - Casa Forte, Recife - PE ',
       ' Rua Doutor Luiz Inácio Pessoa de Melo, 81 - Boa Viagem, Recife - PE '],
      dtype=object)

In [34]:
vivareal['locations'] = vivareal['locations'].apply(lambda x: re.findall("[a-zA-Z]+? ?[a-zA-Z]+? ?[a-zA-Z|ç]+?, Recife", x)[0]
                                                    .split(',')[0])
vivareal.head()

,prices,locations,sqft,bedrooms,bathrooms,garages
0,1700000.0,Casa Forte,207,4,5,3
1,NaN,Cordeiro,55-73,1-3,1,1-2
2,780000.0,Boa Viagem,250,4,4,1
3,450000.0,Casa Forte,82,3,3,2
4,350000.0,Boa Viagem,70,2,2,1


### 3.4 ZAPIMOVEIS
Reaproveitar dados em que trabalhei inicialmente ao decidir fazer um projeto.

In [39]:
df_zapimovies = pd.read_csv('zapimoveis.csv', index_col=0)
df_zapimovies.rename(columns= {'X_bairro': 'locations', 'X_tamanho_em_m2': 'sqft',
                               'X_número_de_quartos': 'bedrooms', 'X_número_de_banheiros': 'bathrooms',
                              'X_vagas_para_estacionar': 'garages', 'y_preço': 'prices'}, inplace = True)
df_zapimovies.drop(columns="Localização" ,inplace=True)
df_zapimovies.head()

,locations,sqft,bedrooms,garages,bathrooms,prices
0,Várzea,69,2,1,1,262900
2,Aflitos,54,2,1,2,450000
3,Imbiribeira,204,4,3,5,2500000
4,Boa Viagem,70,2,1,2,499000
5,Boa Viagem,74,3,1,3,350000


### 3.5 Unindo DataFrames

In [61]:
data = pd.concat([df_olx, df_redeimoveis, vivareal, df_zapimovies], axis=0)
data.head()

,prices,locations,bedrooms,sqft,condominiums,garages,bathrooms
0,750000.0,Boa Viagem,3,130,NaN,1,NaN
1,200000.0,Candeias,2,61,NaN,1,NaN
2,300000.0,Cordeiro,2,50,1,1,NaN
3,128000.0,Nossa Senhora do Ó,2,63,NaN,1,NaN
4,125000.0,Areias,2,62,140,1,NaN


In [62]:
#data.to_pickle("data analysis.pkl")

Agora com os dados limpos podemos fazer uma análise estatística. Utilizar conhecimento do negócio para tratar dos problemas existentes e em seguida preprocessar as informações para uso em um modelo de ML.